<a href="https://colab.research.google.com/github/xxFirefly/mpc_rl_capsubot/blob/f%2Fadd-colab/example_mpc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Example of model usage

In [15]:
#%matplotlib
if 'google.colab' in str(get_ipython()):
  import os
  ! git clone https://github.com/xxFirefly/mpc_rl_capsubot  
  ! pip install casadi==3.5.5
  os.chdir("mpc_rl_capsubot")

from envs.CapsubotEnv import CapsubotEnv
import numpy as np
from time import time
import matplotlib.pyplot as plt
import scipy.constants

# https://github.com/MMehrez/MPC-and-MHE-implementation-in-MATLAB-using-Casadi/blob/master/workshop_github/Python_Implementation/mpc_code.py

# pip install casadi
from casadi import *

try:
    env.close()
except Exception:
    pass

env = CapsubotEnv(force="trivial")

fatal: destination path 'mpc_rl_capsubot' already exists and is not an empty directory.


/usr/local/lib/python3.7/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


# Helper functions

In [ ]:
def shift_timestep(step_horizon, t0, state_init, u, f):
    f_value = f(state_init, u[:, 0])
    next_state = DM.full(state_init + (step_horizon * f_value))

    t0 = t0 + step_horizon
    u0 = horzcat(
        u[:, 1:],
        reshape(u[:, -1], -1, 1)
    )

    return t0, next_state, u0


def DM2Arr(dm):
    return np.array(dm.full())

# State and control variables.

In [ ]:
# Based on the exapmle 
# https://github.com/MMehrez/MPC-and-MHE-implementation-in-MATLAB-using-Casadi/blob/master/workshop_github/Python_Implementation/mpc_code.py

x = SX.sym("x")
xi = SX.sym("xi")
x_dot = SX.sym("x_dot")
xi_dot = SX.sym("xi_dot")
states = vertcat(x, x_dot, xi, xi_dot)
n_states = states.numel()
force = SX.sym("F")
controls = vertcat(force)
n_controls = controls.numel()


# Model

In [ ]:
friction = env.friction_model(x_dot)
x_acc = (env.stiffness * xi - force + friction) / env.M
xi_acc = (-env.stiffness * xi + force) / env.m - x_acc

rhs = vertcat(x_dot, x_acc, xi_dot, xi_acc)  # System r.h.s

# Nonlinear mapping function f(x,u).
f = Function("f", [states, controls], [rhs])

# Simulate data and calculate objective function.

In [ ]:
# setting matrix_weights' variables
Q_x = 100
Q_y = 100
Q_theta = 2000
R1 = 1

F_max = 1.25 # Need to add proper force model
F_min = 0 

x_init = 0
x_target = 5


step_horizon = env.dt
# [s]
N = int(0.1 / env.dt)  # Prediction horizon.

# A vector that represents the states over the optimization problem.
X = SX.sym("X", n_states, (N + 1))
# Decision variables (controls)
U = SX.sym("U", n_controls, N)
# Parameters (which include the initial state and the reference state)
P = SX.sym("P", n_states + n_states)
# state weights matrix (Q_X, Q_Y, Q_THETA)
Q = diagcat(Q_x, Q_y, Q_theta)
# controls weights matrix
R = diagcat(R1)

# OBJECTIVE FUNCTION PARAMETERS.
# Weighing matrices (states).
Q = SX(4, 4)  # FIXME USE OTHER WAY TO CREATE ZERO MATRIX
Q[0, 0] = 1
Q[1, 1] = 1
Q[2, 2] = 1
Q[3, 3] = 1

# Weighing matrices (controls)
R = SX(1, 1)
R[0, 0] = 1


cost_fn = 0  # cost function (objective function?)
g = X[:, 0] - P[:n_states]  # constraints in the equation for multi shooting.

print("prediction_horizon:", N)
# GENERATE STATES AND CONSTRAINTS FOR WHOLE HORIZION
for k in range(N):
    st = X[:, k]
    con = U[:, k]
    cost_fn = cost_fn + (st - P[n_states:]).T @ Q @ (st - P[n_states:]) + con.T @ R @ con
    st_next = X[:, k + 1]
    f_value = f(st, con)
    st_next_euler = st + (step_horizon * f_value)
    g = vertcat(g, st_next - st_next_euler) # Compute constraints.


prediction_horizon: 4000


# Problem solving

In [ ]:
OPT_variables = vertcat(
    X.reshape((-1, 1)), U.reshape((-1, 1))  # Example: 3x11 ---> 33x1 where 3=states, 11=N+1
)
nlp_prob = {"f": cost_fn, "x": OPT_variables, "g": g, "p": P}

opts = {
    "ipopt": {
        "max_iter": 2000,
        "print_level": 0,
        "acceptable_tol": 1e-8,
        "acceptable_obj_change_tol": 1e-6,
    },
    "print_time": 0,
}

solver = nlpsol("solver", "ipopt", nlp_prob, opts)

lbx = DM.zeros((n_states * (N + 1) + n_controls * N, 1))
ubx = DM.zeros((n_states * (N + 1) + n_controls * N, 1))

lbx[0 : n_states * (N + 1) : n_states] = -1  # X lower bound
lbx[1 : n_states * (N + 1) : n_states] = -1  # x_dot lower bound
lbx[2 : n_states * (N + 1) : n_states] = -0.2  # xi lower bound
lbx[3 : n_states * (N + 1) : n_states] = -1  # xi_dot lower bound


ubx[0 : n_states * (N + 1) : n_states] = inf  # X lower bound
ubx[1 : n_states * (N + 1) : n_states] = 1  # x_dot lower bound
ubx[2 : n_states * (N + 1) : n_states] = 0.2  # xi lower bound
ubx[3 : n_states * (N + 1) : n_states] = 1  # xi_dot lower bound


lbx[n_states * (N + 1) :] = F_min  # v lower bound for F.. NEED TO DOBUBLE CHECK THIS
ubx[n_states * (N + 1) :] = F_max  # v upper bound for F


args = {
    "lbg": DM.zeros((n_states * (N + 1), 1)),  # constraints lower bound
    "ubg": DM.zeros((n_states * (N + 1), 1)),  # constraints upper bound
    "lbx": lbx,
    "ubx": ubx,
}


In [ ]:
t0 = 0
state_init = DM([0, 0, 0, 0])  # initial state
state_target = DM([5, 0, 0, 0])  # target state

t = DM(t0)

u0 = DM.zeros((n_controls, N))  # initial control
X0 = repmat(state_init, 1, N + 1)  # initial state full


mpc_iter = 0
cat_states = DM2Arr(X0)
cat_controls = DM2Arr(u0[:, 0])
times = np.array([[0]])


In [ ]:
sim_time = 10
main_loop = time()  # return time in sec
while (norm_2(state_init - state_target) > 1e-1) and (mpc_iter * step_horizon < sim_time):
    t1 = time()
    args["p"] = vertcat(state_init, state_target)  # current state  # target state
    # optimization variable current state
    args["x0"] = vertcat(reshape(X0, n_states * (N + 1), 1), reshape(u0, n_controls * N, 1))

    sol = solver(
        x0=args["x0"],
        lbx=args["lbx"],
        ubx=args["ubx"],
        lbg=args["lbg"],
        ubg=args["ubg"],
        p=args["p"],
    )

    u = reshape(sol["x"][n_states * (N + 1) :], n_controls, N)
    X0 = reshape(sol["x"][: n_states * (N + 1)], n_states, N + 1)

    cat_states = np.dstack((cat_states, DM2Arr(X0)))

    cat_controls = np.vstack((cat_controls, DM2Arr(u[:, 0])))
    t = np.vstack((t, t0))

    t0, state_init, u0 = shift_timestep(step_horizon, t0, state_init, u, f)

    print(X0)
    X0 = horzcat(X0[:, 1:], reshape(X0[:, -1], -1, 1))

    # xx ...
    t2 = time()
    if (mpc_iter * step_horizon) % 0.005 <= step_horizon:
        print("iter n:", mpc_iter)
        print("iter time:", t2 - t1)
        print("simulated time:", mpc_iter * step_horizon)
        print("pose error:", norm_2(state_init - state_target))
        print(state_init)
    print("iter n:", mpc_iter)
    print("iter time:", t2 - t1)
    print("simulated time:", mpc_iter * step_horizon)
    print("pose error:", norm_2(state_init - state_target))
    print(state_init)

    times = np.vstack((times, t2 - t1))

    mpc_iter = mpc_iter + 1

main_loop_time = time()
ss_error = ca.norm_2(state_init - state_target)

print("\n\n")
print("Total time: ", main_loop_time - main_loop)
print("avg iteration time: ", np.array(times).mean() * 1000, "ms")
print("final error: ", ss_error)



In [ ]:
states = np.array(states)

fig, axs = plt.subplots(2, 2, sharex=True)  # , sharex=True, sharey=True)
fig.suptitle("average speed: {:.3}".format(states[-1, 0] / ts[-1]))
axs[0, 0].plot(ts, states[:, 0], label="x")
axs[0, 0].set_title("x")
axs[1, 0].plot(ts, states[:, 1], label="dx")
axs[1, 0].set_title("dx")
axs[0, 1].plot(ts, states[:, 2] + states[:, 0], label="xi")
axs[1, 0].set_title("xi")
axs[1, 1].plot(ts, states[:, 3], label="dxi")
axs[1, 1].set_title("dxi")
t_shift = 3
axs[0, 0].set_xlim(t_shift, t_shift + 1)


plt.figure()
plt.plot(ts, actions, label="F")
plt.xlim(t_shift, t_shift + 1)
plt.legend()
plt.show()



NameError: name 'np' is not defined